In [1]:
import numpy as np

In [3]:
import pandas as pd
df = pd.read_csv('bot_human_answer.csv')

In [4]:
df

,Question,Human Answer,Bot Answer
0,What versions of the package are vulnerable to...,"The package forms before 1.2.1, from 1.3.0, an...","The package forms before 1.2.1, from 1.3.0 and..."
1,What vulnerability was identified in SnapGear ...,denial of service (complete packet loss) via a...,Remote attackers could cause a denial of servi...
2,What infrastructure was used to carry out the ...,C&C servers and email addresses.,The attackers used a compromised command-and-c...
3,Who were the primary victims targeted in the M...,The disclosed victims included a high-profile ...,The primary victims targeted in the Mirage cam...
4,What is the name of the tool used by the Naiko...,The tool used by the Naikon APT group for remo...,The tool used by the Naikon APT group for remo...
5,What type of malware infected the Turkish airl...,The prevalent and powerful info-stealing troja...,The malware that infected the Turkish airline ...
6,What are some aliases of the Jupyter Infosteal...,"Polazert, SolarMarker, and Yellow Cockatoo","Polazert, SolarMarker, and Yellow Cockatoo."
7,When did the attack on Kaseya take place?,The attack on Kaseya occurred over the July 4 ...,The attack on Kaseya took place on the July 4 ...
8,List the different versions of Win32/Injector....,"26690, 26977, and 26603",The different versions of Win32/Injector.EDSK ...
9,List the different version of Trojan.Win32.PRI...,10.0.0.1040 and 11.0.0.1006.,The different versions of Trojan.Win32.PRIVATE...


In [5]:
questions=df['Question'].to_list()

In [6]:
answers=df['Human Answer'].to_list()

In [7]:
bot_answer=df['Bot Answer'].to_list()

In [8]:
import pandas as pd
import numpy as np
from scipy import spatial
from transformers import BertTokenizer, BertModel
import torch

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [9]:
# Function to tokenize and encode sentences with BERT
def encode_sentence(sentence, model, tokenizer):
    if isinstance(sentence, str):
        tokens = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=128, padding='max_length')
        input_ids = torch.tensor(tokens).unsqueeze(0)  # Batch size 1
        with torch.no_grad():
            outputs = model(input_ids)
            last_hidden_states = outputs.last_hidden_state  # Last layer hidden-states
        return last_hidden_states.mean(dim=1).squeeze().numpy()
    else:
        # Handle NaN or non-string inputs
        return np.zeros(768)  # Return a default value or appropriate handling


In [10]:
# Compute cosine similarity using BERT embeddings
S1 = []
for i in range(len(answers)):
    sentence1 = answers[i]
    sentence2 = bot_answer[i]

    # Encode sentences using BERT
    s1_emb = encode_sentence(sentence1, model, tokenizer)
    s2_emb = encode_sentence(sentence2, model, tokenizer)

    # Compute cosine similarity
    sim = 1 - spatial.distance.cosine(s1_emb, s2_emb)
    S1.append(sim)
    print(f"Cosine similarity between the sentences: {sim}")


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Cosine similarity between the sentences: 0.9983369708061218
Cosine similarity between the sentences: 0.9792910814285278
Cosine similarity between the sentences: 0.9076599478721619
Cosine similarity between the sentences: 0.9639638662338257
Cosine similarity between the sentences: 0.9965171813964844
Cosine similarity between the sentences: 0.9962120652198792
Cosine similarity between the sentences: 0.9987086653709412
Cosine similarity between the sentences: 0.9996585845947266
Cosine similarity between the sentences: 0.9773806929588318
Cosine similarity between the sentences: 0.9699668884277344


In [11]:
df['BERT'] = S1

In [12]:
# Save the DataFrame to a new CSV file
df.to_csv('bot_human_answer.csv', index=False)

In [13]:
df

,Question,Human Answer,Bot Answer,BERT
0,What versions of the package are vulnerable to...,"The package forms before 1.2.1, from 1.3.0, an...","The package forms before 1.2.1, from 1.3.0 and...",0.998337
1,What vulnerability was identified in SnapGear ...,denial of service (complete packet loss) via a...,Remote attackers could cause a denial of servi...,0.979291
2,What infrastructure was used to carry out the ...,C&C servers and email addresses.,The attackers used a compromised command-and-c...,0.907660
3,Who were the primary victims targeted in the M...,The disclosed victims included a high-profile ...,The primary victims targeted in the Mirage cam...,0.963964
4,What is the name of the tool used by the Naiko...,The tool used by the Naikon APT group for remo...,The tool used by the Naikon APT group for remo...,0.996517
5,What type of malware infected the Turkish airl...,The prevalent and powerful info-stealing troja...,The malware that infected the Turkish airline ...,0.996212
6,What are some aliases of the Jupyter Infosteal...,"Polazert, SolarMarker, and Yellow Cockatoo","Polazert, SolarMarker, and Yellow Cockatoo.",0.998709
7,When did the attack on Kaseya take place?,The attack on Kaseya occurred over the July 4 ...,The attack on Kaseya took place on the July 4 ...,0.999659
8,List the different versions of Win32/Injector....,"26690, 26977, and 26603",The different versions of Win32/Injector.EDSK ...,0.977381
9,List the different version of Trojan.Win32.PRI...,10.0.0.1040 and 11.0.0.1006.,The different versions of Trojan.Win32.PRIVATE...,0.969967
